In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
path_to_csv = "/content/drive/MyDrive/UIT - Semester 6/Data Analysis in Business/Dataset"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, explained_variance_score
from xgboost import XGBRegressor
import laggedfeature

sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")

ModuleNotFoundError: ignored

In [ ]:
df = pd.read_csv(path_to_csv + "/TSLA.csv", usecols=['Date', 'Close'])
df['Date'] = pd.to_datetime(df['Date'])
df['MonthYear'] = df['Date'].dt.year.astype(str) +'-'+df['Date'].dt.month.astype(str)
df['MonthYear'] = df['MonthYear'].drop_duplicates()
df.dropna()
df.reset_index(drop=True)
df

In [ ]:
# Create lagged feature
num_lags = 7
df['Lag'] = laggedfeature.set(num_lags, df['Close'])

In [ ]:
# Train test val split in 7-2-1 set
test_size = int(len(df)*0.20)
val_size = int(len(df)*0.10)
train_size = int(len(df)-test_size-val_size)

train_data = df[:train_size]
test_data = df[train_size:train_size+test_size]
val_data = df[train_size+test_size:]

In [ ]:
model = XGBRegressor(**{'learning_rate':0.4})
model.fit(train_data['Lag'], train_data['Close'])
model.score(test_data['Lag'], test_data['Close'])

In [ ]:
y_test_pred = model.predict(test_data['Lag'])
y_val_pred = model.predict(val_data['Lag'])

In [ ]:
# RMSE
rmse = mean_squared_error(test_data['Close'], y_test_pred, squared=False)
print("The root mean squared error (RMSE) on test set: {:.3f}".format(rmse))

In [ ]:
#MAPE
mape = mean_absolute_percentage_error(test_data['Close'], y_test_pred)
print("The mean absolute percentage error (MAPE) on test set: {:.3f}".format(mape))

In [ ]:
# EVS
evs = explained_variance_score(test_data['Close'], y_test_pred)
print("The explained variance score (EVS) on test set: {:.3f}".format(evs))

In [ ]:
# Predicting next 30 days' prices
last_index=df.index[-1]

X_30 = laggedfeature.set_30(num_lags, df['Close'])
X_30.index = pd.RangeIndex(start=last_index+1, stop=last_index+31)
y_30_pred = model.predict(X_30)

In [ ]:
plt.subplots(figsize=(12, 4))
plt.plot(train_data.index, train_data['Close'])
plt.plot(test_data.index, test_data['Close'])
plt.plot(test_data.index, y_test_pred)
plt.plot(val_data.index, val_data['Close'])
plt.plot(val_data.index, y_val_pred)
plt.plot(X_30.index, y_30_pred)
plt.legend(['Train', 'Test', 'Predictions', 'Validate',
           'Validate Pred', 'Next 30 Days Pred'])

temp = df.dropna(subset=['MonthYear'])
temp = temp['MonthYear'].loc[::3]
plt.xticks(temp.index, temp, rotation=45)

plt.xlabel('Date')
plt.show()

In [ ]:
# import datetime

# last_day = df['Date'].iloc[-1]
# next_days = np.array([])

# for i in range(0, 30):
#     last_day = last_day+datetime.timedelta(days=1)
#     next_days = np.append(next_days, last_day)

plt.plot(y_30_pred)
# plt.xticks(next_days[::2], rotation=45)
plt.show()